# `SVM` with `NB` features

This notebook implements shows how to use the NB-SVM classifier from [this paper](https://nlp.stanford.edu/pubs/sidaw12_simple_sentiment.pdf) that has become a classic (and competitive) baseline.

## toy dataset (binary)

Working example borrowed from [Stanford IR online text](https://nlp.stanford.edu/IR-book/html/htmledition/naive-bayes-text-classification-1.html)

| docID | text                                | label     |
|-------|-------------------------------------|-----------|
| 1     | chinese beijing chinese             | CHINA     |
| 2     | chinese chinese shanghai            | CHINA     |
| 3     | chinese macao                       | CHINA     |
| 4     | tokyo japan chinese                 | NOT_CHINA |
| 5     | chinese chinese chinese tokyo japan | ???       |


In [1]:
doc_one = "chinese beijing chinese"
label_one = 1

doc_two = "chinese chinese shanghai"
label_two = 1

doc_three = "chinese macao"
label_three = 1

doc_four = "tokyo japan chinese"
label_four = 0

all_docs = [doc_one, doc_two, doc_three, doc_four]
all_labels = [label_one, label_two, label_three, label_four]

In [2]:
from collections import OrderedDict
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import *
from sklearn.linear_model import *
from sklearn.naive_bayes import *
from sklearn.ensemble import *
from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics import accuracy_score
import numpy as np
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

import inspect
import os
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
os.sys.path.insert(0, os.path.join(parentdir, 'nb_transformer'))
from nb_transformer import NaiveBayesTransformer
from nb_enhanced_classifier import NaiveBayesEnhancedClassifier

## traditional `bernoulli` `vectorizer`

First let's `vectorize` the documents in the training set using `CountVectorizer()`. <br>
The paper found binary features to be more effective than raw counts.

In [3]:
vectorizer_bernoulli = CountVectorizer(
    binary=True
)
X_bernoulli = vectorizer_bernoulli.fit_transform(all_docs)

In [4]:
vectorizer_bernoulli.vocabulary_

{'beijing': 0, 'chinese': 1, 'japan': 2, 'macao': 3, 'shanghai': 4, 'tokyo': 5}

In [5]:
X_bernoulli.toarray()

array([[1, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 1, 0],
       [0, 1, 0, 1, 0, 0],
       [0, 1, 1, 0, 0, 1]], dtype=int64)

## `NaiveBayesTransformer`

This is a custom `transformer` that will use the Naive Bayes conditional probabilities as feature values.  See [the paper](https://nlp.stanford.edu/pubs/sidaw12_simple_sentiment.pdf) for details.

The key thing to recognize is that this method is built off of the assumption of a `binary` (only two labels) problem.  It can be expanded to `multiclass` utilizing the `one-v-all` approach, but this also requires a **different** transformation for **each** label, thus the inclusion of `label_of_interest` as an argument to the `NaiveBayesTransformer`.

In [6]:
nb_transformer = NaiveBayesTransformer(all_labels, 1)

### original feature space

In [7]:
original_features = vectorizer_bernoulli.fit_transform(all_docs)
original_features.toarray()

array([[1, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 1, 0],
       [0, 1, 0, 1, 0, 0],
       [0, 1, 1, 0, 0, 1]], dtype=int64)

### transformed feature space

In [8]:
nb_transformed = nb_transformer.fit_transform(original_features)
nb_transformed

array([[ 0.4054651 ,  0.4054651 , -0.        ,  0.        ,  0.        ,
        -0.        ],
       [ 0.        ,  0.4054651 , -0.        ,  0.        ,  0.4054651 ,
        -0.        ],
       [ 0.        ,  0.4054651 , -0.        ,  0.4054651 ,  0.        ,
        -0.        ],
       [ 0.        ,  0.4054651 , -0.98082924,  0.        ,  0.        ,
        -0.98082924]])

## Train

Given our features, let's train a classifier to predict `1` v. `0`. 
The paper found an `SVM` with `squared loss` and `l2` penalizing to be most effective.

In [9]:
svm_clf = LinearSVC(            
    loss='squared_hinge',
    penalty='l2',
    random_state = 1,         # to ensure reproducible results
    class_weight='balanced',
    dual=False
)

Let's build an instance of `NaiveBayesEnchancedClassifier`.

In [10]:
svm_clf.get_params()

{'C': 1.0,
 'class_weight': 'balanced',
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'loss': 'squared_hinge',
 'max_iter': 1000,
 'multi_class': 'ovr',
 'penalty': 'l2',
 'random_state': 1,
 'tol': 0.0001,
 'verbose': 0}

In [12]:
binary_example_clf = NaiveBayesEnhancedClassifier(
    clf=svm_clf
)

Let's train it on our dataset by feeding the *original* vectorized features into `.fit()`

In [13]:
binary_example_clf.fit(original_features, all_labels)

Baked into `.fit()` are the following steps:
 1. `transform` the features using the `NaiveBayesTransformer`
 2. Apply an interpolation of weights to the `.coef_` of the `classifier` using the formula below: 
  - $w' = (1 - \beta)\bar{w} + \beta w$ where $\bar{w}$ is the mean magnitude of all the weights
  
Below are a few cells showing the calculations that are done in `NaiveBayesTransformer._interpolate()`

In [14]:
svm_clf.coef_

array([[ 0.3158278 ,  0.32634704,  0.45491727,  0.3158278 ,  0.3158278 ,
         0.45491727]])

In [15]:
interpolation_factor = 0.25

In [16]:
mean_magnitude = np.sum(svm_clf.coef_)/svm_clf.coef_.shape[1]
mean_magnitude_vector = np.full(svm_clf.coef_.shape, mean_magnitude)
mean_magnitude_vector

array([[ 0.36394416,  0.36394416,  0.36394416,  0.36394416,  0.36394416,
         0.36394416]])

In [17]:
svm_clf.coef_ = (1 - interpolation_factor) * mean_magnitude_vector + \
                                interpolation_factor * svm_clf.coef_
svm_clf.coef_

array([[ 0.35191507,  0.35454488,  0.38668744,  0.35191507,  0.35191507,
         0.38668744]])

## Test

Now let's predict on a new data point.

In [18]:
test_doc = "chinese chinese chinese tokyo japan"

We first must `vectorize` the test item into the original feature space.

In [19]:
test_features_original = vectorizer_bernoulli.transform([test_doc])

Then we can use `NaiveBayesEnhancedClassifier.predict()` to (1) apply the `NB-transformation` and make our prediction.

In [20]:
binary_example_clf.predict(test_features_original)

array([0])

We can see the "confidence" of this prediction using `NaiveBayesEnhancedClassifier.decision_function_predict_proba(test_features_original)` which will use `.decision_function()` or `.predict_proba()` of the original `classifier`.

In [21]:
binary_example_clf.decision_function_predict_proba(test_features_original)

array([-0.08810002])

The model predicts the test document to be in the `not China` class but not with much confidence.  

Note: This is opposite the prediction made in the Stanford IR worked example because they use `multinomial NB` and the presence of "China" three times outweighed the presence of the other words.

## `multiclass` test



As explained above, this transformation is built for `binary` classification.  The `multiclass` version requires the application of `one-v-all` to a traditional `SVM` classifier.  This is all done automatically within the `NaiveBayesEnhancedClassifier` `class`.  An example is below.

In [22]:
obvious_docs = [
    "cat",
    "cat",
    "cat",
    "pencil",
    "pencil",
    "pencil",
    "car",
    "car",
    "car",
]

obvious_labels = [
    0,
    0,
    0,
    1,
    1,
    1,
    2,
    2,
    2
]

test_docs = [
    "cat",
    "pen",
    "pencil",
    "car"
]

### traditional `bernoulli` `vectorizer`

In [23]:
multiclass_bernoulli_vectorizer = CountVectorizer(
    binary=True
)

multiclass_original_features = multiclass_bernoulli_vectorizer.fit_transform(obvious_docs)

In [24]:
multiclass_original_features.toarray()

array([[0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0]], dtype=int64)

### Train

In [28]:
multiclass_example_clf = NaiveBayesEnhancedClassifier(
    # we'll use the default settings for `clf`
)

In [29]:
multiclass_example_clf.fit(
    multiclass_bernoulli_vectorizer.fit_transform(obvious_docs),   # vectorize the original documents
    obvious_labels
)

### Test

Let's project the test documents into the same feature space as the training data.

In [30]:
multiclass_test_features = multiclass_bernoulli_vectorizer.transform(test_docs)
multiclass_test_features.toarray()

array([[0, 1, 0],
       [0, 0, 0],
       [0, 0, 1],
       [1, 0, 0]])

Then let's get predictions for each test document.

In [31]:
multiclass_example_clf.predict(multiclass_test_features)

array([0, 0, 1, 2])

Behind the scenes, the classifier is generating predictions for how likely the data point is in that class.  We can see these values by calling `NaiveBayesEnhancedClassifier.decision_function_predict_proba()`.

Below, each row is a classifier trained for a different label.  The values in the vector represent the margin of each data point from the decision boundary.

In [32]:
for i, row in zip([0,1,2], multiclass_example_clf.decision_function_predict_proba(multiclass_test_features)):
    print(i, row)

0 [ 0.48603642 -0.4641514  -0.98429487 -1.06914084]
1 [-0.98429487 -0.4641514   0.48603642 -1.06914084]
2 [-0.98429487 -0.4641514  -0.98429487  0.64103137]


We then select the "most confident" classifier for each data point, where "most confident" is the one with the largest *positive* (or, if they're all *negative*, the *smallest*) margin.  In this case, each column is a data point, so we take the `argmax` of each column.

Note: Since the second data point ("pen") was never seen in the training vocabulary, the classifiers all generate the same prediction, none of which are positive (implying a low level of confidence)

### using `classifier` without `.coef_` or `.decision_function()`

The `NaiveBayesEnhancedClassifier` is built to wrap *any* `sklearn` classifier.  If the classifier does *not* use weights that are captured in the attribute, `.coef_`, then `interpolation` is skipped.

In [33]:
multiclass_example_no_decision_function = NaiveBayesEnhancedClassifier(
    clf=RandomForestClassifier()
)

In [34]:
multiclass_example_no_decision_function.fit(
    multiclass_bernoulli_vectorizer.fit_transform(obvious_docs),   # vectorize the original documents
    obvious_labels
)

/media/mcapizzi/data/Github/naive-bayes-transformer/nb_transformer/nb_enhanced_classifier.py:172: UserWarning:

the classifier you instantiated does not have an attribute `.coef_`; interpolation will not occur



Likewise, if the `classifier` chosen does *not* have a `.decision_function()` method, then `.predict_proba()` is used.

In [36]:
multiclass_example_no_decision_function.decision_function_predict_proba(multiclass_test_features)

/media/mcapizzi/data/Github/naive-bayes-transformer/nb_transformer/nb_enhanced_classifier.py:234: UserWarning:

classifier you instantiated does not have a method, `decision_function()`;using `predict_proba()` instead



array([[ 0.3,  0.3,  0. ,  0.3],
       [ 0. ,  0.3,  0.3,  0.3],
       [ 0. ,  0.3,  0. ,  1. ]])

In this case, each row still represents a `binary` classifier for a given label, and the vector values are the probabilities of the `positive` class.  We still choose the `argmax` in each column to determine our prediction.

In [37]:
multiclass_example_no_decision_function.predict(multiclass_test_features)

/media/mcapizzi/data/Github/naive-bayes-transformer/nb_transformer/nb_enhanced_classifier.py:234: UserWarning:

classifier you instantiated does not have a method, `decision_function()`;using `predict_proba()` instead



array([0, 0, 1, 2])

# Experiments on `20Newsgroups`

Below we attempt to recreate the `binary` classification problems in the experiments of the [original paper](https://nlp.stanford.edu/pubs/sidaw12_simple_sentiment.pdf).  Whenever provided, we use the same hyperparameters.

We also compare different classifiers and the use of (1) no transformer or (2) `TfidfVectorizer()`.

Note: It's likely that the data coming from `sklearn` for `20Newsgroups` is *not* exactly the same as the data used by the authors.  But the results are comparable.

## `alt.atheism` v. `talk.religion.misc` (`AthR`)

In [ ]:
categories = [
    'alt.atheism', 'talk.religion.misc',
]

In [ ]:
remove = (
    'headers'
)

## Data

In [ ]:
data_train = fetch_20newsgroups(subset='train', categories=categories,
                                shuffle=True, random_state=42,
                                remove=remove)

data_test = fetch_20newsgroups(subset='test', categories=categories,
                               shuffle=True, random_state=42,
                               remove=remove)

idx2class_lookup = dict((i,c) for i,c in enumerate(data_train.target_names))
idx2class_lookup

In [ ]:
# get label distribution
num_pos = list(data_train.target).count(1)
num_neg = list(data_train.target).count(0)
num_pos, num_neg

### Vectorize

In [ ]:
newsgroups_vectorizer_uni = CountVectorizer(
    binary=True
)

newsgroups_vectorizer_tfidf = TfidfVectorizer()

In [ ]:
newsgroups_train_features_uni = newsgroups_vectorizer_uni.fit_transform(data_train.data)
newsgroups_train_features_uni

In [ ]:
newsgroups_test_features_uni = newsgroups_vectorizer_uni.transform(data_test.data)
newsgroups_test_features_uni

### transformed - nb

In [ ]:
all_classifiers = OrderedDict()

all_classifiers["svm_from_paper"] = LinearSVC(
    loss='squared_hinge',
    penalty='l2',
    random_state=1,
    dual=False
)

all_classifiers["sgd"] = SGDClassifier(
    n_iter=50,
    penalty='elasticnet',
    random_state=1,
)

all_classifiers["ridge"] = RidgeClassifier(
    tol=1e-2,
    solver='lsqr',
    random_state=1,
)

all_classifiers["perceptron"] = Perceptron(
    n_iter=50,
    random_state=1,
)

all_classifiers["passive_aggressive"] = PassiveAggressiveClassifier(
    n_iter=50,
    random_state=1,
)

all_classifiers["random_forest"] = RandomForestClassifier(
    n_estimators=100,
    random_state=1,
)

all_classifiers["adaboost"] = AdaBoostClassifier(
    n_estimators=100,
    random_state=1,
)

In [ ]:
newsgroups_nb_plus_clfs = OrderedDict()
for clf_name, clf in all_classifiers.items():
    newsgroups_nb_plus_clfs[clf_name] = NaiveBayesEnhancedClassifier(
        list(map(lambda x: x[0], idx2class_lookup.items())),
        clf,
        interpolation_factor=None
)

In [ ]:
# transformed - nb
for clf_name, clf in newsgroups_nb_plus_clfs.items():
    print("training {} - nb transformed".format(clf_name))
    clf.fit(newsgroups_train_features_uni, data_train.target)

In [ ]:
newsgroups_nb_transformed_uni_results = OrderedDict()
for clf_name, clf in newsgroups_nb_plus_clfs.items():
    print("testing transformed {}".format(clf_name))
    preds = clf.predict(newsgroups_test_features_uni)
    newsgroups_nb_transformed_uni_results[clf_name] = accuracy_score(data_test.target, preds)
newsgroups_nb_transformed_uni_results

### no transformation

In [ ]:
all_classifiers = OrderedDict()

all_classifiers["svm_from_paper"] = LinearSVC(
    loss='squared_hinge',
    penalty='l2',
    dual=False,
    C=0.1,
    random_state=1
)

all_classifiers["sgd"] = SGDClassifier(
    n_iter=50,
    penalty='elasticnet',
    random_state=1,
)

all_classifiers["ridge"] = RidgeClassifier(
    tol=1e-2,
    solver='lsqr',
    random_state=1,
)

all_classifiers["perceptron"] = Perceptron(
    n_iter=50,
    random_state=1,
)

all_classifiers["passive_aggressive"] = PassiveAggressiveClassifier(
    n_iter=50,
    random_state=1,
)

all_classifiers["random_forest"] = RandomForestClassifier(
    n_estimators=100,
    random_state=1,
)

all_classifiers["adaboost"] = AdaBoostClassifier(
    n_estimators=100,
    random_state=1,
)

all_classifiers["bernoulli_nb"] = BernoulliNB(
    alpha=0.01,
)

all_classifiers["multinomial_nb"] = MultinomialNB(
    alpha=0.01,
)

In [ ]:
# no transformation count
for clf_name, clf in all_classifiers.items():
    print("training baseline {} - no transformation".format(clf_name))
    clf.fit(newsgroups_train_features_uni, data_train.target)

In [ ]:
newsgroups_no_transformation_uni_results = OrderedDict()
for clf_name, clf in all_classifiers.items():
    print("testing baseline {} - no transformation".format(clf_name))
    preds = clf.predict(newsgroups_test_features_uni)
    newsgroups_no_transformation_uni_results[clf_name] = accuracy_score(data_test.target, preds)
newsgroups_no_transformation_uni_results

### transformation - tfidf

In [ ]:
newsgroups_train_features_uni = newsgroups_vectorizer_tfidf.fit_transform(data_train.data)
newsgroups_test_features_uni = newsgroups_vectorizer_tfidf.transform(data_test.data)

In [ ]:
all_classifiers = OrderedDict()

all_classifiers["svm_from_paper"] = LinearSVC(
    loss='squared_hinge',
    penalty='l2',
    C=0.1,
    random_state=1,
)

all_classifiers["sgd"] = SGDClassifier(
    n_iter=50,
    penalty='elasticnet',
    random_state=1,
)

all_classifiers["ridge"] = RidgeClassifier(
    tol=1e-2,
    solver='lsqr',
    random_state=1,
)

all_classifiers["perceptron"] = Perceptron(
    n_iter=50,
    random_state=1,
)

all_classifiers["passive_aggressive"] = PassiveAggressiveClassifier(
    n_iter=50,
    random_state=1,
)

all_classifiers["random_forest"] = RandomForestClassifier(
    n_estimators=100,
    random_state=1,
)

all_classifiers["adaboost"] = AdaBoostClassifier(
    n_estimators=100,
    random_state=1,
)

all_classifiers["bernoulli_nb"] = BernoulliNB(
    alpha=0.01
)

all_classifiers["multinomial_nb"] = MultinomialNB(
    alpha=0.01
)

In [ ]:
for clf_name, clf in all_classifiers.items():
    print("training baseline {} - tfidf transformation".format(clf_name))
    clf.fit(newsgroups_train_features_uni, data_train.target)

In [ ]:
newsgroups_tfidf_transformed_uni_results = OrderedDict()
for clf_name, clf in all_classifiers.items():
    print("testing baseline {} - no transformation".format(clf_name))
    preds = clf.predict(newsgroups_test_features_uni)
    newsgroups_tfidf_transformed_uni_results[clf_name] = accuracy_score(data_test.target, preds)
newsgroups_tfidf_transformed_uni_results

In [ ]:
groups = [
    ("no_transform", newsgroups_no_transformation_uni_results),
    ("tfidf_transform", newsgroups_tfidf_transformed_uni_results), 
    ("nb_transform", newsgroups_nb_transformed_uni_results)
]

clfs = list(all_classifiers.keys())

traces = OrderedDict()
for i in range(len(groups)):
    transformation = groups[i][1]
    transformation_name = groups[i][0]
    scores = []
    for label in clfs:
        try:
            score = transformation[label]
        except:
            score = 0.0
        scores.append(score)
    traces["trace_{}".format(transformation_name)] = go.Bar(
        x=clfs,
        y=scores,
        name=transformation_name,
    )

data_ = [v for k, v in traces.items()]
layout_ = go.Layout(
    barmode='group',
    title='Transformation comparison: AthR (2.9)',
    yaxis=dict(
        range=[0, 1]
    )
)

fig_ = go.Figure(data=data_, layout=layout_)
iplot(fig_)    

## `comp.graphics` v. `comp.windows.x` (`XGraph`)

In [ ]:
categories = [
    'comp.graphics', 'comp.windows.x',
]

## Data

In [ ]:
data_train = fetch_20newsgroups(subset='train', categories=categories,
                                shuffle=True, random_state=42,
                                remove=remove)

data_test = fetch_20newsgroups(subset='test', categories=categories,
                               shuffle=True, random_state=42,
                               remove=remove)

idx2class_lookup = dict((i,c) for i,c in enumerate(data_train.target_names))
idx2class_lookup

In [ ]:
# get label distribution
num_pos = list(data_train.target).count(1)
num_neg = list(data_train.target).count(0)
num_pos, num_neg

## Unigrams

### Vectorize

In [ ]:
newsgroups_vectorizer_uni = CountVectorizer(
    binary=True
)

newsgroups_vectorizer_tfidf = TfidfVectorizer()

In [ ]:
newsgroups_train_features_uni = newsgroups_vectorizer_uni.fit_transform(data_train.data)
newsgroups_train_features_uni

In [ ]:
newsgroups_test_features_uni = newsgroups_vectorizer_uni.transform(data_test.data)
newsgroups_test_features_uni

### transformed - nb

In [ ]:
all_classifiers = OrderedDict()

all_classifiers["svm_from_paper"] = LinearSVC(
    loss='squared_hinge',
    penalty='l2',
    random_state=1,
    dual=False
)

all_classifiers["sgd"] = SGDClassifier(
    n_iter=50,
    penalty='elasticnet',
    random_state=1,
)

all_classifiers["ridge"] = RidgeClassifier(
    tol=1e-2,
    solver='lsqr',
    random_state=1,
)

all_classifiers["perceptron"] = Perceptron(
    n_iter=50,
    random_state=1,
)

all_classifiers["passive_aggressive"] = PassiveAggressiveClassifier(
    n_iter=50,
    random_state=1,
)

all_classifiers["random_forest"] = RandomForestClassifier(
    n_estimators=100,
    random_state=1,
)

all_classifiers["adaboost"] = AdaBoostClassifier(
    n_estimators=100,
    random_state=1,
)

In [ ]:
newsgroups_nb_plus_clfs = OrderedDict()
for clf_name, clf in all_classifiers.items():
    newsgroups_nb_plus_clfs[clf_name] = NaiveBayesEnhancedClassifier(
        list(map(lambda x: x[0], idx2class_lookup.items())),
        clf,
        interpolation_factor=None
)

In [ ]:
# transformed - nb
for clf_name, clf in newsgroups_nb_plus_clfs.items():
    print("training {} - nb transformed".format(clf_name))
    clf.fit(newsgroups_train_features_uni, data_train.target)

In [ ]:
newsgroups_nb_transformed_uni_results = OrderedDict()
for clf_name, clf in newsgroups_nb_plus_clfs.items():
    print("testing transformed {}".format(clf_name))
    preds = clf.predict(newsgroups_test_features_uni)
    newsgroups_nb_transformed_uni_results[clf_name] = accuracy_score(data_test.target, preds)
newsgroups_nb_transformed_uni_results

### no transformation

In [ ]:
all_classifiers = OrderedDict()

all_classifiers["svm_from_paper"] = LinearSVC(
    loss='squared_hinge',
    penalty='l2',
    dual=False,
    C=0.1,
    random_state=1
)

all_classifiers["sgd"] = SGDClassifier(
    n_iter=50,
    penalty='elasticnet',
    random_state=1,
)

all_classifiers["ridge"] = RidgeClassifier(
    tol=1e-2,
    solver='lsqr',
    random_state=1,
)

all_classifiers["perceptron"] = Perceptron(
    n_iter=50,
    random_state=1,
)

all_classifiers["passive_aggressive"] = PassiveAggressiveClassifier(
    n_iter=50,
    random_state=1,
)

all_classifiers["random_forest"] = RandomForestClassifier(
    n_estimators=100,
    random_state=1,
)

all_classifiers["adaboost"] = AdaBoostClassifier(
    n_estimators=100,
    random_state=1,
)

all_classifiers["bernoulli_nb"] = BernoulliNB(
    alpha=0.01,
)

all_classifiers["multinomial_nb"] = MultinomialNB(
    alpha=0.01,
)

In [ ]:
# no transformation count
for clf_name, clf in all_classifiers.items():
    print("training baseline {} - no transformation".format(clf_name))
    clf.fit(newsgroups_train_features_uni, data_train.target)

In [ ]:
newsgroups_no_transformation_uni_results = OrderedDict()
for clf_name, clf in all_classifiers.items():
    print("testing baseline {} - no transformation".format(clf_name))
    preds = clf.predict(newsgroups_test_features_uni)
    newsgroups_no_transformation_uni_results[clf_name] = accuracy_score(data_test.target, preds)
newsgroups_no_transformation_uni_results

### transformation - tfidf

In [ ]:
newsgroups_train_features_uni = newsgroups_vectorizer_tfidf.fit_transform(data_train.data)
newsgroups_test_features_uni = newsgroups_vectorizer_tfidf.transform(data_test.data)

In [ ]:
all_classifiers = OrderedDict()

all_classifiers["svm_from_paper"] = LinearSVC(
    loss='squared_hinge',
    penalty='l2',
    C=0.1,
    random_state=1,
)

all_classifiers["sgd"] = SGDClassifier(
    n_iter=50,
    penalty='elasticnet',
    random_state=1,
)

all_classifiers["ridge"] = RidgeClassifier(
    tol=1e-2,
    solver='lsqr',
    random_state=1,
)

all_classifiers["perceptron"] = Perceptron(
    n_iter=50,
    random_state=1,
)

all_classifiers["passive_aggressive"] = PassiveAggressiveClassifier(
    n_iter=50,
    random_state=1,
)

all_classifiers["random_forest"] = RandomForestClassifier(
    n_estimators=100,
    random_state=1,
)

all_classifiers["adaboost"] = AdaBoostClassifier(
    n_estimators=100,
    random_state=1,
)

all_classifiers["bernoulli_nb"] = BernoulliNB(
    alpha=0.01
)

all_classifiers["multinomial_nb"] = MultinomialNB(
    alpha=0.01
)

In [ ]:
for clf_name, clf in all_classifiers.items():
    print("training baseline {} - tfidf transformation".format(clf_name))
    clf.fit(newsgroups_train_features_uni, data_train.target)

In [ ]:
newsgroups_tfidf_transformed_uni_results = OrderedDict()
for clf_name, clf in all_classifiers.items():
    print("testing baseline {} - no transformation".format(clf_name))
    preds = clf.predict(newsgroups_test_features_uni)
    newsgroups_tfidf_transformed_uni_results[clf_name] = accuracy_score(data_test.target, preds)
newsgroups_tfidf_transformed_uni_results

In [ ]:
groups = [
    ("no_transform", newsgroups_no_transformation_uni_results),
    ("tfidf_transform", newsgroups_tfidf_transformed_uni_results), 
    ("nb_transform", newsgroups_nb_transformed_uni_results)
]

clfs = list(all_classifiers.keys())

traces = OrderedDict()
for i in range(len(groups)):
    transformation = groups[i][1]
    transformation_name = groups[i][0]
    scores = []
    for label in clfs:
        try:
            score = transformation[label]
        except:
            score = 0.0
        scores.append(score)
    traces["trace_{}".format(transformation_name)] = go.Bar(
        x=clfs,
        y=scores,
        name=transformation_name,
    )

data_ = [v for k, v in traces.items()]
layout_ = go.Layout(
    barmode='group',
    title='Transformation comparison: XGraph (1.8)',
    yaxis=dict(
        range=[0, 1]
    )
)

fig_ = go.Figure(data=data_, layout=layout_)
iplot(fig_)    

## `rec.sport.baseball` v. `sci.crypt` (`BbCrypt`)

In [ ]:
categories = [
    'rec.sport.baseball', 'sci.crypt',
]

## Data

In [ ]:
data_train = fetch_20newsgroups(subset='train', categories=categories,
                                shuffle=True, random_state=42,
                                remove=remove)

data_test = fetch_20newsgroups(subset='test', categories=categories,
                               shuffle=True, random_state=42,
                               remove=remove)

idx2class_lookup = dict((i,c) for i,c in enumerate(data_train.target_names))
idx2class_lookup

In [ ]:
# get label distribution
num_pos = list(data_train.target).count(1)
num_neg = list(data_train.target).count(0)
num_pos, num_neg

In [ ]:
# determine intercept (log(N+/N-))
intercept = np.log(num_pos/num_neg)
intercept

## Unigrams

### Vectorize

In [ ]:
newsgroups_vectorizer_uni = CountVectorizer(
    binary=True
)

newsgroups_vectorizer_tfidf = TfidfVectorizer()

In [ ]:
newsgroups_train_features_uni = newsgroups_vectorizer_uni.fit_transform(data_train.data)
newsgroups_train_features_uni

In [ ]:
newsgroups_test_features_uni = newsgroups_vectorizer_uni.transform(data_test.data)
newsgroups_test_features_uni

### transformed - nb

In [ ]:
all_classifiers = OrderedDict()

all_classifiers["svm_from_paper"] = LinearSVC(
    loss='squared_hinge',
    penalty='l2',
    random_state=1,
    dual=False
)

all_classifiers["sgd"] = SGDClassifier(
    n_iter=50,
    penalty='elasticnet',
    random_state=1,
)

all_classifiers["ridge"] = RidgeClassifier(
    tol=1e-2,
    solver='lsqr',
    random_state=1,
)

all_classifiers["perceptron"] = Perceptron(
    n_iter=50,
    random_state=1,
)

all_classifiers["passive_aggressive"] = PassiveAggressiveClassifier(
    n_iter=50,
    random_state=1,
)

all_classifiers["random_forest"] = RandomForestClassifier(
    n_estimators=100,
    random_state=1,
)

all_classifiers["adaboost"] = AdaBoostClassifier(
    n_estimators=100,
    random_state=1,
)

In [ ]:
newsgroups_nb_plus_clfs = OrderedDict()
for clf_name, clf in all_classifiers.items():
    newsgroups_nb_plus_clfs[clf_name] = NaiveBayesEnhancedClassifier(
        list(map(lambda x: x[0], idx2class_lookup.items())),
        clf,
        interpolation_factor=None
)

In [ ]:
# transformed - nb
for clf_name, clf in newsgroups_nb_plus_clfs.items():
    print("training {} - nb transformed".format(clf_name))
    clf.fit(newsgroups_train_features_uni, data_train.target)

In [ ]:
newsgroups_nb_transformed_uni_results = OrderedDict()
for clf_name, clf in newsgroups_nb_plus_clfs.items():
    print("testing transformed {}".format(clf_name))
    preds = clf.predict(newsgroups_test_features_uni)
    newsgroups_nb_transformed_uni_results[clf_name] = accuracy_score(data_test.target, preds)
newsgroups_nb_transformed_uni_results

### no transformation

In [ ]:
all_classifiers = OrderedDict()

all_classifiers["svm_from_paper"] = LinearSVC(
    loss='squared_hinge',
    penalty='l2',
    dual=False,
    C=0.1,
    random_state=1
)

all_classifiers["sgd"] = SGDClassifier(
    n_iter=50,
    penalty='elasticnet',
    random_state=1,
)

all_classifiers["ridge"] = RidgeClassifier(
    tol=1e-2,
    solver='lsqr',
    random_state=1,
)

all_classifiers["perceptron"] = Perceptron(
    n_iter=50,
    random_state=1,
)

all_classifiers["passive_aggressive"] = PassiveAggressiveClassifier(
    n_iter=50,
    random_state=1,
)

all_classifiers["random_forest"] = RandomForestClassifier(
    n_estimators=100,
    random_state=1,
)

all_classifiers["adaboost"] = AdaBoostClassifier(
    n_estimators=100,
    random_state=1,
)

all_classifiers["bernoulli_nb"] = BernoulliNB(
    alpha=0.01,
)

all_classifiers["multinomial_nb"] = MultinomialNB(
    alpha=0.01,
)

In [ ]:
# no transformation count
for clf_name, clf in all_classifiers.items():
    print("training baseline {} - no transformation".format(clf_name))
    clf.fit(newsgroups_train_features_uni, data_train.target)

In [ ]:
newsgroups_no_transformation_uni_results = OrderedDict()
for clf_name, clf in all_classifiers.items():
    print("testing baseline {} - no transformation".format(clf_name))
    preds = clf.predict(newsgroups_test_features_uni)
    newsgroups_no_transformation_uni_results[clf_name] = accuracy_score(data_test.target, preds)
newsgroups_no_transformation_uni_results

### transformation - tfidf

In [ ]:
newsgroups_train_features_uni = newsgroups_vectorizer_tfidf.fit_transform(data_train.data)
newsgroups_test_features_uni = newsgroups_vectorizer_tfidf.transform(data_test.data)

In [ ]:
all_classifiers = OrderedDict()

all_classifiers["svm_from_paper"] = LinearSVC(
    loss='squared_hinge',
    penalty='l2',
    C=0.1,
    random_state=1,
)

all_classifiers["sgd"] = SGDClassifier(
    n_iter=50,
    penalty='elasticnet',
    random_state=1,
)

all_classifiers["ridge"] = RidgeClassifier(
    tol=1e-2,
    solver='lsqr',
    random_state=1,
)

all_classifiers["perceptron"] = Perceptron(
    n_iter=50,
    random_state=1,
)

all_classifiers["passive_aggressive"] = PassiveAggressiveClassifier(
    n_iter=50,
    random_state=1,
)

all_classifiers["random_forest"] = RandomForestClassifier(
    n_estimators=100,
    random_state=1,
)

all_classifiers["adaboost"] = AdaBoostClassifier(
    n_estimators=100,
    random_state=1,
)

all_classifiers["bernoulli_nb"] = BernoulliNB(
    alpha=0.01
)

all_classifiers["multinomial_nb"] = MultinomialNB(
    alpha=0.01
)

In [ ]:
for clf_name, clf in all_classifiers.items():
    print("training baseline {} - tfidf transformation".format(clf_name))
    clf.fit(newsgroups_train_features_uni, data_train.target)

In [ ]:
newsgroups_tfidf_transformed_uni_results = OrderedDict()
for clf_name, clf in all_classifiers.items():
    print("testing baseline {} - no transformation".format(clf_name))
    preds = clf.predict(newsgroups_test_features_uni)
    newsgroups_tfidf_transformed_uni_results[clf_name] = accuracy_score(data_test.target, preds)
newsgroups_tfidf_transformed_uni_results

In [ ]:
groups = [
    ("no_transform", newsgroups_no_transformation_uni_results),
    ("tfidf_transform", newsgroups_tfidf_transformed_uni_results), 
    ("nb_transform", newsgroups_nb_transformed_uni_results)
]

clfs = list(all_classifiers.keys())

traces = OrderedDict()
for i in range(len(groups)):
    transformation = groups[i][1]
    transformation_name = groups[i][0]
    scores = []
    for label in clfs:
        try:
            score = transformation[label]
        except:
            score = 0.0
        scores.append(score)
    traces["trace_{}".format(transformation_name)] = go.Bar(
        x=clfs,
        y=scores,
        name=transformation_name,
    )

data_ = [v for k, v in traces.items()]
layout_ = go.Layout(
    barmode='group',
    title='Transformation comparison: BbCrypt (0.5)',
    yaxis=dict(
        range=[0, 1]
    )
)

fig_ = go.Figure(data=data_, layout=layout_)
iplot(fig_)    

## `20Newsgroups`

In [ ]:
categories = None

## Data

In [ ]:
data_train = fetch_20newsgroups(subset='train', categories=categories,
                                shuffle=True, random_state=42,
                                remove=remove)

data_test = fetch_20newsgroups(subset='test', categories=categories,
                               shuffle=True, random_state=42,
                               remove=remove)

idx2class_lookup = dict((i,c) for i,c in enumerate(data_train.target_names))
idx2class_lookup

## Unigrams

### Vectorize

In [ ]:
newsgroups_vectorizer_uni = CountVectorizer(
    binary=True
)

newsgroups_vectorizer_tfidf = TfidfVectorizer()

In [ ]:
newsgroups_train_features_uni = newsgroups_vectorizer_uni.fit_transform(data_train.data)
newsgroups_train_features_uni

In [ ]:
newsgroups_test_features_uni = newsgroups_vectorizer_uni.transform(data_test.data)
newsgroups_test_features_uni

### transformed - nb

In [ ]:
all_classifiers = OrderedDict()

all_classifiers["svm_from_paper"] = LinearSVC(
    loss='squared_hinge',
    penalty='l2',
    random_state=1,
    dual=False
)

all_classifiers["sgd"] = SGDClassifier(
    n_iter=50,
    penalty='elasticnet',
    random_state=1,
)

all_classifiers["ridge"] = RidgeClassifier(
    tol=1e-2,
    solver='lsqr',
    random_state=1,
)

all_classifiers["perceptron"] = Perceptron(
    n_iter=50,
    random_state=1,
)

all_classifiers["passive_aggressive"] = PassiveAggressiveClassifier(
    n_iter=50,
    random_state=1,
)

all_classifiers["random_forest"] = RandomForestClassifier(
    n_estimators=100,
    random_state=1,
)

all_classifiers["adaboost"] = AdaBoostClassifier(
    n_estimators=100,
    random_state=1,
)

In [ ]:
newsgroups_nb_plus_clfs = OrderedDict()
for clf_name, clf in all_classifiers.items():
    newsgroups_nb_plus_clfs[clf_name] = NaiveBayesEnhancedClassifier(
        list(map(lambda x: x[0], idx2class_lookup.items())),
        clf,
        interpolation_factor=None
)

In [ ]:
# transformed - nb
for clf_name, clf in newsgroups_nb_plus_clfs.items():
    print("training {} - nb transformed".format(clf_name))
    clf.fit(newsgroups_train_features_uni, data_train.target)

In [ ]:
newsgroups_nb_transformed_uni_results = OrderedDict()
for clf_name, clf in newsgroups_nb_plus_clfs.items():
    print("testing transformed {}".format(clf_name))
    preds = clf.predict(newsgroups_test_features_uni)
    newsgroups_nb_transformed_uni_results[clf_name] = accuracy_score(data_test.target, preds)
newsgroups_nb_transformed_uni_results

In [ ]:
newsgroups_nb_transformed_uni_results

### no transformation

In [ ]:
all_classifiers = OrderedDict()

all_classifiers["svm_from_paper"] = LinearSVC(
    loss='squared_hinge',
    penalty='l2',
    dual=False,
    C=0.1,
    random_state=1
)

all_classifiers["sgd"] = SGDClassifier(
    n_iter=50,
    penalty='elasticnet',
    random_state=1,
)

all_classifiers["ridge"] = RidgeClassifier(
    tol=1e-2,
    solver='lsqr',
    random_state=1,
)

all_classifiers["perceptron"] = Perceptron(
    n_iter=50,
    random_state=1,
)

all_classifiers["passive_aggressive"] = PassiveAggressiveClassifier(
    n_iter=50,
    random_state=1,
)

all_classifiers["random_forest"] = RandomForestClassifier(
    n_estimators=100,
    random_state=1,
)

all_classifiers["adaboost"] = AdaBoostClassifier(
    n_estimators=100,
    random_state=1,
)

all_classifiers["bernoulli_nb"] = BernoulliNB(
    alpha=0.01,
)

all_classifiers["multinomial_nb"] = MultinomialNB(
    alpha=0.01,
)

In [ ]:
# no transformation count
for clf_name, clf in all_classifiers.items():
    print("training baseline {} - no transformation".format(clf_name))
    clf.fit(newsgroups_train_features_uni, data_train.target)

In [ ]:
newsgroups_no_transformation_uni_results = OrderedDict()
for clf_name, clf in all_classifiers.items():
    print("testing baseline {} - no transformation".format(clf_name))
    preds = clf.predict(newsgroups_test_features_uni)
    newsgroups_no_transformation_uni_results[clf_name] = accuracy_score(data_test.target, preds)
newsgroups_no_transformation_uni_results

### transformation - tfidf

In [ ]:
newsgroups_train_features_uni = newsgroups_vectorizer_tfidf.fit_transform(data_train.data)
newsgroups_test_features_uni = newsgroups_vectorizer_tfidf.transform(data_test.data)

In [ ]:
all_classifiers = OrderedDict()

all_classifiers["svm_from_paper"] = LinearSVC(
    loss='squared_hinge',
    penalty='l2',
    C=0.1,
    random_state=1,
)

all_classifiers["sgd"] = SGDClassifier(
    n_iter=50,
    penalty='elasticnet',
    random_state=1,
)

all_classifiers["ridge"] = RidgeClassifier(
    tol=1e-2,
    solver='lsqr',
    random_state=1,
)

all_classifiers["perceptron"] = Perceptron(
    n_iter=50,
    random_state=1,
)

all_classifiers["passive_aggressive"] = PassiveAggressiveClassifier(
    n_iter=50,
    random_state=1,
)

all_classifiers["random_forest"] = RandomForestClassifier(
    n_estimators=100,
    random_state=1,
)

all_classifiers["adaboost"] = AdaBoostClassifier(
    n_estimators=100,
    random_state=1,
)

all_classifiers["bernoulli_nb"] = BernoulliNB(
    alpha=0.01
)

all_classifiers["multinomial_nb"] = MultinomialNB(
    alpha=0.01
)

In [ ]:
for clf_name, clf in all_classifiers.items():
    print("training baseline {} - tfidf transformation".format(clf_name))
    clf.fit(newsgroups_train_features_uni, data_train.target)

In [ ]:
newsgroups_tfidf_transformed_uni_results = OrderedDict()
for clf_name, clf in all_classifiers.items():
    print("testing baseline {} - no transformation".format(clf_name))
    preds = clf.predict(newsgroups_test_features_uni)
    newsgroups_tfidf_transformed_uni_results[clf_name] = accuracy_score(data_test.target, preds)
newsgroups_tfidf_transformed_uni_results

In [ ]:
groups = [
    ("no_transform", newsgroups_no_transformation_uni_results),
    ("tfidf_transform", newsgroups_tfidf_transformed_uni_results), 
    ("nb_transform", newsgroups_nb_transformed_uni_results)
]

clfs = list(all_classifiers.keys())

traces = OrderedDict()
for i in range(len(groups)):
    transformation = groups[i][1]
    transformation_name = groups[i][0]
    scores = []
    for label in clfs:
        try:
            score = transformation[label]
        except:
            score = 0.0
        scores.append(score)
    traces["trace_{}".format(transformation_name)] = go.Bar(
        x=clfs,
        y=scores,
        name=transformation_name,
    )

data_ = [v for k, v in traces.items()]
layout_ = go.Layout(
    barmode='group',
    title='Transformation comparison: 20Newsgroups',
    yaxis=dict(
        range=[0, 1]
    )
)

fig_ = go.Figure(data=data_, layout=layout_)
iplot(fig_)    